I am going to attempt to develop a model that will predict who will win any individal mens ncaa college basketball game. Then, I would like to see how accuratly the model can predict Michigan States record from the 2019 season. I chose 2019 because MSU made the final four so it seemed like the most exciting option.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

I got the data from https://www.sports-reference.com/cbb/seasons/men/2019-ratings.html

In [17]:
data = pd.read_csv("cbb_2019.csv", skiprows = 1)
data = data.drop("Unnamed: 0", axis = 1)
data = data.drop('Rk', axis = 1)

In [18]:
data

,School,G,W,L,W-L%,SRS,SOS,MP,FG,FGA,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian NCAA,34,27,7,0.794,-1.91,-7.34,1370,897,1911,...,457,642,0.712,325,1110,525,297,93,407,635
1,Air Force,32,14,18,0.438,-4.28,0.24,1300,802,1776,...,341,503,0.678,253,1077,434,154,57,423,543
2,Akron,33,17,16,0.515,4.86,1.09,1325,797,1948,...,380,539,0.705,312,1204,399,185,106,388,569
3,Alabama,34,18,16,0.529,9.45,9.01,1365,856,1945,...,492,739,0.666,384,1285,418,157,160,465,572
4,Alabama A&M,32,5,27,0.156,-19.23,-8.38,1295,736,1809,...,284,453,0.627,314,1032,385,234,50,487,587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Wright State,35,21,14,0.600,3.29,-0.89,1405,885,2028,...,510,692,0.737,382,1229,484,214,72,402,545
349,Wyoming,32,8,24,0.250,-9.75,0.19,1285,690,1656,...,477,660,0.723,167,983,331,176,88,450,588
350,Xavier,35,19,16,0.543,9.61,8.06,1420,922,1977,...,437,644,0.679,371,1281,519,190,128,450,550
351,Yale NCAA,30,22,8,0.733,5.52,-1.24,1215,893,1812,...,411,557,0.738,259,1157,503,177,131,392,510


In [22]:
data2 = pd.read_csv("cbb19.csv")

I got the data from https://barttorvik.com/trank.php?year=2021&sort=&top=0&conlimit=All#

In [24]:
data2 = data2.drop_duplicates(subset = "TEAM", keep="first")

In [25]:
data2.fillna({'SEED':'Missed Tournament'}, inplace = True)
data2.SEED.value_counts(dropna=False)
data2.fillna({'POSTSEASON':'Missed Tournament'}, inplace = True)
data2.SEED.value_counts(dropna=False)

Missed Tournament    285
16.0                   6
11.0                   6
10.0                   4
15.0                   4
14.0                   4
13.0                   4
12.0                   4
1.0                    4
2.0                    4
8.0                    4
7.0                    4
6.0                    4
5.0                    4
4.0                    4
3.0                    4
9.0                    4
Name: SEED, dtype: int64

In [26]:
data2

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED
0,Gonzaga,WCC,37,33,123.4,89.9,0.9744,59.0,44.2,14.9,...,35.3,25.9,61.4,43.4,36.3,30.4,72.0,7.0,E8,1.0
1,Virginia,ACC,38,35,123.0,89.9,0.9736,55.2,44.7,14.7,...,29.1,26.3,52.5,45.7,39.5,28.9,60.7,11.1,Champions,1.0
2,Duke,ACC,38,32,118.9,89.2,0.9646,53.6,45.0,17.5,...,33.2,24.0,58.0,45.0,30.8,29.9,73.6,11.2,E8,1.0
3,North Carolina,ACC,36,29,120.1,91.4,0.9582,52.9,48.9,17.2,...,30.2,28.4,52.1,47.9,36.2,33.5,76.0,10.0,S16,1.0
4,Michigan,B10,37,30,114.6,85.6,0.9665,51.6,44.1,13.9,...,27.5,24.1,51.8,44.3,34.2,29.1,65.9,9.2,S16,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Alcorn St.,SWAC,27,10,89.0,112.6,0.0628,45.7,52.7,24.1,...,30.5,36.5,45.0,55.3,31.3,32.1,67.1,-16.7,Missed Tournament,Missed Tournament
349,New Hampshire,AE,27,5,83.7,106.1,0.0613,44.0,51.5,18.4,...,21.9,38.0,39.4,52.1,32.6,33.6,67.1,-20.2,Missed Tournament,Missed Tournament
350,Chicago St.,WAC,30,3,88.5,117.3,0.0380,44.2,57.8,22.5,...,33.1,33.9,43.5,57.9,30.7,38.5,71.9,-20.9,Missed Tournament,Missed Tournament
351,Delaware St.,MEAC,29,6,84.3,112.2,0.0358,40.0,52.4,19.0,...,25.5,39.2,37.7,52.6,29.0,34.7,71.6,-21.7,Missed Tournament,Missed Tournament


After doing some research on how to best predict the score of a basketball game, I came up with the following alogrithm. First, I will find the number of possessions per game that a team averages. This is because some teams like to play faster and will therefore have more possesions per game, while other teams take their time and utilize the entire shot clock, resulting in less possesions per game. To estimate the number of possessions in the game from the box score, let’s consider how a possession can end.

One way for a possession to end is a turnover. Then we look at a teams field goal attempts. This can lead to three different outcomes: the offense makes the shot, the offense misses the shot and the defense gets the rebound, or the team gets an offensive rebound which would result in a new possession. 

A possessions can also end with free throws. If every free throw came in a pair because of a shooting foul, the number of possessions would be half the free throw attempts. Unfortunately that’s not always the case, as the offense can:

get one free throw after a made basket
get three free throws when fouled on a three pointer
miss the front end of a one-and-one

Instead of a half, you need a different factor that accounts for these situations. I found that instead of using 1/2, to best esitmate this, I should use 0.475, so that is what I did. 

To estimate possessions (POSS) from the box score:

POSS = FGA – OREB + TO + (0.475 * FTA)

Next, I need to find each teams adjusted offensive efficency, which is the amount of points a team would score per 100 possessions, or trips down the floor with the basketball, against an average Division I opponent. Defensive efficency works the same way, except it is the amount of points an average opponent would score on them. 

After I have that I needed to find each teams points per possession. To do this, I multiplied the teams offensive efficency by the other teams defensive efficency and divided that by the leauge average.

Lastly, I multiplied the points per possession by the number of possessions and divided by 100. 

The only thing I did not account for is home court advantage, because not every team would have played in real life, and I want to use the algorithm to predict for teams that may have not played in the 2019 season.

In [155]:
def game_score(team1, team2):
    val = data.index[data['School']==team1].tolist()
    idx = data.index[data['School']==team2].tolist()
    val = val[0]
    idx = idx[0]
    poss_team1 = (data.loc[val]['FGA']-data.loc[val]['ORB']+data.loc[val]['TOV']+(0.475*data.loc[val]['FTA']))/(data.loc[val]['G'])
    poss_team2= (data.loc[idx]['FGA']-data.loc[idx]['ORB']+data.loc[idx]['TOV']+(0.475*data.loc[idx]['FTA']))/(data.loc[idx]['G'])
    adjoe_team1 = data2.loc[val]['ADJOE']
    adjoe_team2 = data2.loc[idx]['ADJOE']
    adjde_team1 = data2.loc[val]['ADJDE']
    adjde_team2 = data2.loc[idx]['ADJDE']
    
    leauge_ave_off = np.mean(data2['ADJOE'])
    
    ppp_team1 = (adjoe_team1*adjde_team2)/(leauge_ave_off)
    ppp_team2 = (adjoe_team2*adjde_team1)/(leauge_ave_off)
    
    score_team1 = (ppp_team1*poss_team1)/100
    score_team2 = (ppp_team2*poss_team2)/100
    
    return(score_team1, score_team2)

    


In [161]:
def winner(team1, team2):
    final = game_score(team1, team2)
    points_team1 = final[0]
    points_team2 = final[1]
    if(points_team1 > points_team2):
        winner = team1
    if(points_team2 > points_team1):
        winner = team2
    return(winner)

In [216]:
winner('Oral Roberts', 'Loyola (IL)')

'Oral Roberts'

Now, I want to see how accuratly my algorithm can worked against Michigan States actual schedule for 2019. Below is MSU's schedule.

In [219]:
msu_2019_schedule = ['Kansas NCAA', 'Florida Gulf Coast', 'Louisiana-Monroe', 'Tennessee Tech', 'UCLA', 'Texas', 'Louisville NCAA', 'Rutgers', 'Iowa NCAA', 'Florida NCAA', 'Green Bay', 'Oakland', 'Northern Illinois', 'Northwestern', 'Ohio State NCAA', 'Purdue NCAA', 'Penn State', 'Nebraska', 'Maryland NCAA', 'Iowa NCAA', 'Purdue NCAA', 'Indiana', 'Illinois', 'Minnesota NCAA', 'Wisconsin NCAA', 'Ohio State NCAA', 'Rutgers', 'Michigan NCAA', 'Indiana', 'Nebraska', 'Michigan NCAA']
msu_outcomes = []
for i in range(len(msu_2019_schedule)):
    hold = winner('Michigan State NCAA', msu_2019_schedule[i])
    msu_outcomes.append(hold)

In [220]:
msu_outcomes

['Kansas NCAA',
 'Florida Gulf Coast',
 'Louisiana-Monroe',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Louisville NCAA',
 'Michigan State NCAA',
 'Iowa NCAA',
 'Florida NCAA',
 'Green Bay',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Iowa NCAA',
 'Michigan State NCAA',
 'Indiana',
 'Illinois',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Michigan State NCAA',
 'Indiana',
 'Michigan State NCAA',
 'Michigan State NCAA']

Based on this, my algorithm did pretty well. In reality, in the regular season, MSU won 25 games instead of 20. The incorrectly predicted games were Florida Gulf Coast, Louisina-Monroe, Green Bay, and both times against Iowa. I think for the Florida Gulf Coast, Louisiana-Monroe, and Green Bay games, it is because those teams are "mid-majors" meaning that they are not in a power 5 conference (Big10, Big12, SEC, Pac12, and ACC). They would have played other mid majors most of the time, which would have benefited their statistics. 